In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts


In [16]:
#CA VA APPARAITRE EN PREMIER POUR CHOSIR UN SECTOR
sectors = [
    'Communication_Services', 
    'Energy', 
    'financials', 
    'Health_Care', 
    'Industrials', 
    'Information_Technology', 
    'Real_Estate'
]

#function 1 : gives the data and their selected stocks of each stock (it wont be in the front it will be just the data provider of the other functions bellow) 
def load_sector_data(sector):
    """
    Function to load data for a specific sector.
    
    :param sector: A string representing the sector to load data for.
    :return: A tuple where the first element is a DataFrame with the selected stocks data, and the second element is a DataFrame with the price data.
    """
    # Load the selected stocks financials and sentiment data from the CSV file
    selected_stocks_df = pd.read_csv(f'selected_stocks_{sector}.csv', index_col='symbol')

    # Load the price data
    price_data = pd.read_csv(f'hist adjust close {sector}.csv', index_col='Date', parse_dates=True)
    
    return selected_stocks_df, price_data


#function 2 : gives the stocks of each cluster of the selected sector (it will be in the front in a table format to show the stocks of each cluster in each sector)

def cluster_stocks(sector):
    """
    Cluster the stocks of a specific sector based on their adjusted correlation.
    
    :param sector: A string representing the sector to cluster stocks for.
    """
    # Load the data for the sector
    selected_stocks_df, price_data = load_sector_data(sector)

    # Calculate returns and drop NA values
    price_returns = price_data.pct_change().dropna()

    # Calculate the correlation matrix
    correlation_matrix = price_returns.corr()

    # Adjust the correlation matrix here as needed before clustering

    # Cluster the stocks based on the correlation matrix using the k-means algorithm
    n_clusters = 4  # Set the number of clusters
    kmeans = KMeans(n_clusters=n_clusters)
    cluster_labels = kmeans.fit_predict(correlation_matrix)

    # Create a dictionary mapping stock symbols to cluster labels
    symbol_cluster_dict = {symbol: cluster_labels[i] for i, symbol in enumerate(price_returns.columns)}
    
    # Create a dictionary to store the tickers for each cluster
    cluster_dict = {}
    for i in range(n_clusters):
        cluster_dict[f'Cluster {i+1}'] = price_returns.columns[cluster_labels==i].tolist()
    return cluster_dict


#function 3 : gives the visualisation of the kmeans algo of stocks of the selected sector (it will be in the front in visualisation format)

def visualize_clusters(sector):
    """
    Cluster the stocks of a specific sector and visualize the clusters.
    
    :param sector: A string representing the sector to cluster stocks for.
    """
    # Get the clusters and the price returns for the sector
    clusters_dict = cluster_stocks(sector)

    price_returns=load_sector_data(sector)[1].pct_change().dropna()
    # Get the number of clusters
    n_clusters = len(clusters_dict)

    # Visualize the clustering results
    fig, ax = plt.subplots(figsize=(18, 12))

    # Set point size and color based on cluster labels
    point_size = 150
    colors = ['red', 'blue', 'green', 'orange']
    for i, cluster_label in enumerate(clusters_dict.keys()):
        cluster_stocks_list = clusters_dict[cluster_label]
        ax.scatter(price_returns[cluster_stocks_list].mean(), price_returns[cluster_stocks_list].std(), s=point_size, color=colors[i], alpha=0.7, label=cluster_label)

        # Add stock labels to each point
        for symbol in cluster_stocks_list:
            ax.annotate(symbol, xy=(price_returns[symbol].mean(), price_returns[symbol].std()), 
                        xytext=(5,-5), textcoords='offset points', fontsize=12)

    ax.set_xlabel('Expected Returns', fontsize=16)
    ax.set_ylabel('Risk', fontsize=16)
    ax.set_title(f'{n_clusters}-Cluster Solution for {sector} Sector', fontsize=20)
    ax.grid(True, linestyle='--', linewidth=0.5)
    ax.legend(fontsize=14)
    plt.show()


#function 4 : saves the csv files of the clusters generated of the kmeans(it will be usefull for the next part of the trading process the results bcz we are gonna call those csv to backtest the strategies)

def save_clusters_to_csv(sector):
    """
    Cluster the stocks of a specific sector and save each cluster to a separate CSV file.
    
    :param sector: A string representing the sector to cluster stocks for.
    """
    # Get the clusters for the sector
    clusters_dict = cluster_stocks(sector)

    # Save each cluster to a CSV file
    for cluster_label, stocks_list in clusters_dict.items():
        # Save tickers to CSV
        with open(f'{sector}_{cluster_label}.csv', 'w') as f:
            f.write(','.join(stocks_list))

In [33]:

#I WORKED HERE WITH THE Real_Estate AS AN EXEMPLE SINON IL VA PRENDRE VALEUR DIAL SECTOR GIVEN BY THE USER
sector = 'Real_Estate'


"""
the use of function 1
"""

# Load the selected stocks financials and sentiment data from the CSV file
selected_stocks_df = load_sector_data(sector)[0] 

price_data = load_sector_data(sector)[1] 



"""
the use of function 2
"""
# Run the clustering process for the 'Energy' sector
cluster_stocks(sector)



"""
the use of function 3
"""

visualize_clusters(sector)



"""
the use of function 4
"""

save_clusters_to_csv(sector)



{'Cluster 1': ['ARE',
  'AVB',
  'CPT',
  'EQR',
  'ESS',
  'PEAK',
  'INVH',
  'MAA',
  'O',
  'UDR'],
 'Cluster 2': ['AMT', 'CCI', 'DLR', 'EQIX', 'EXR', 'PLD', 'PSA', 'SBAC'],
 'Cluster 3': ['BXP', 'FRT', 'HST', 'KIM', 'REG', 'SPG', 'WELL'],
 'Cluster 4': ['CBRE', 'IRM', 'VICI', 'WY']}